# 14 이메일 프로그래밍

이 장에서는 gmail이나 naver, daum 등의 기존 이메일 시스템을 이용해 메일을 보내고 받는 방법을 설명한다. 텍스트 이메일을 보내는 법, 첨부 파일 메일을 보내는 법 등을 설명한다. 또한 IMAP을 이용하여 메일을 자동으로 읽고 첨부 파일을 다운 받는 법을 설명한다.


## 14.1 이메일 보내기

### 14.1.1 SMTP 서버 연결

인터넷에서 이메일을 보내는 프로토콜로 널리 사용되고 있는 SMTP(Simple Mail Transfer Protocol)는 이름에서 알 수 있듯이 메일을 보내는 프로토콜이다. 쉽게 말하자면 SMTP는 메일을 수신자의 메일 상자에 배달하는 우체국과 같은 역할을 한다. 

SMTP는 초기에는 TCP/UDP 포트 25번을 사용하였으나(1982, RFC-821) 기존 프로토콜을 확장하여 새로운 정책과 보안 요구를 수용하여 587번 포트를 사용하도록 정의했다(1998, RFC-2476). 대부분의 이메일 서버와 클라이언트가 이 포트를 사용하고 있다.  IANA(Internet Assigned Numbers Authority)에서는 SSL(Secure Sockets Layer)을 사용하는 465 포트를 SMTPS로 등록한다. 일부 업체가 465 포트를 사용하고 있다. 

주요 업체의 SMTP 서버 주소와 포트는 다음과 같다.

* 구글 : `smtp.gmail.com`, 포트 587
* 구글 : `smtp.gmail.com`, 포트 465
* 네이버 : `smtp.naver.com`, 포트 587
* 네이트온: `smtp.mail.nate.com`, 포트 465
* 한메일 : `smtp.hanmail.net`, 포트 465
* Hotmail, Outlook : `smtp-mail.outlook.com`, 포트 587
* 야후 : `smtp.mail.yahoo.com`, 포트 587

### 14.1.2 포트 587에 접속하는 방법

`smtplib` 모듈을 이용하면 쉽게 사용할 수 있다.

In [1]:
import smtplib

Gmail인 경우 SMTP를 위해 `smtp.gmail.com` 서버의 587 포트를 사용하고 있다. 다음과 같이 접속 한다.

In [2]:
smtp = smtplib.SMTP('smtp.gmail.com', 587)

접속 이후에 가장 먼저 보내야할 명령은 `ehlo()`이다.

In [3]:
smtp.ehlo()

(250,
 b'smtp.gmail.com at your service, [211.189.163.249]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8')

반환 값이 250이면 성공이다.

접속 포트가 587이면 TSL 암호화를 이용해야 한다. `starttls()` 메서드를 호출해준다.

In [4]:
smtp.starttls()

(220, b'2.0.0 Ready to start TLS')

반환 값이 220이면 서버가 준비되었다는 것을 의미한다.
이제 로그인이 가능하다.

In [5]:
smtp.login('gslee0115@gmail.com', 'mypassword_here')

(235, b'2.7.0 Accepted')

성공 했다면 반환 값이 235이다.

### 14.1.3 포트 465에 접속하는 방법

포트 465를 사용하는 방법도 587을 사용하는 방법과 거의 유사하지만 첫 접속을 `SMTP_SSL()`을 이용해야 하고, `starttls()`은 필요하지 않다.

In [6]:
import smtplib

smtp = smtplib.SMTP_SSL('smtp.gmail.com', 465)

In [7]:
smtp.ehlo()

(250,
 b'smtp.gmail.com at your service, [223.194.27.51]\nSIZE 35882577\n8BITMIME\nAUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8')

In [8]:
smtp.login('gslee0115@gmail.com', 'mypassword_here')

(235, b'2.7.0 Accepted')

### 14.1.4 (Help) 연결이 안될 경우

#### Gmail 설정

    SMTPAuthenticationError: (534, b'5.7.14 <https://accounts.google.com/ContinueSignIn?sarp=1&scc=1&plt=AKgnsbvWh\n5.7.14 sHEN9K6tLpXU3RJD0DKYNP6n2L8npzPJ04YLsi7CGYw6sWhrL9ROgxUXFgfe8P48LsvyGu\n5.7.14 qCG8BQDBhxLE1cqwB2Wcx6GfyV1IuJ5_QJDvHAMVNsGZY7EQRHSVjvemT7P_HvlyHsvSPr\n5.7.14 iUXUxgdBZ-sLuN7Qh-CF2rYc_i4EZ5-UCysT1OnM1HiYxI2S6VPV6Hkvhl05Nna4mqTu8u\n5.7.14 hBKi0hZ7b4qJiGbf8-t7Ur5mut5U> Please log in via your web browser and\n5.7.14 then try again.\n5.7.14  Learn more at\n5.7.14  https://support.google.com/mail/answer/78754 dd4sm13324778pbb.52 - gsmtp')

Google blocks sign-in attempts from apps which do not use modern security standards (mentioned on their support page). You can however, turn on/off this safety feature by going to the link below:

Go to this link and select Turn On
https://www.google.com/settings/security/lesssecureapps

<img src="img/less_secure.png" width="500">

#### 네이버/다음 메일 설정

1. 로그인 한다
1. 메일 페이지로 이동한다
1. 환경 설정 아이콘(기어모양)을 클릭한다
1. IMAP/SMTP 설정 탭 클릭
1. IMAP/SMTP 사용함으로 선택

### 14.1.5 텍스트 메일 보내기


In [9]:
import smtplib
import email
from email.mime.text import MIMEText
from email.header import Header
import getpass
import re

smtp_info = {      # SMTP 서버
    'gmail.com': ('smtp.gmail.com', 587),
    'naver.com': ('smtp.naver.com', 587),
    'hanmail.net': ('smtp.hanmail.net', 465),
    'nate.com': ('smtp.mail.nate.com', 465),
    'hotmail.com': ('smtp-mail.outlook.com', 587),
    'outlook.com': ('smtp-mail.outlook.com', 587),
    'yahoo.com': ('smtp.mail.yahoo.com', 587),
    }

email_pat = r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)"

def sendTextEmail(myemail, receivers, subject, message, passwd=None):
    if type(myemail) is tuple:
        myemail = email.utils.formataddr(myemail)
    sender = re.search(email_pat, myemail).groups()[0]
    mail_to = []
    for receiver in receivers:
        if type(receiver) is tuple:
            receiver = email.utils.formataddr(receiver)
        mail_to.append(receiver)

    host = sender.split('@')[1]
    smtp_server, port = smtp_info[host]
    
    if port == 465:
        smtp = smtplib.SMTP_SSL(smtp_server, port)
        rcode1, _ = smtp.ehlo()
        rcode2 = 220
    elif port == 587:
        smtp = smtplib.SMTP(smtp_server, port)
        rcode1, _ = smtp.ehlo()
        rcode2, _ = smtp.starttls()
    else:
        return 'port {} not suported'.format(port)
        
    if rcode1 != 250:
        smtp.quit()
        return 'conection ehlo() failed'
    if rcode2 != 220:
        smtp.quit()
        return 'starttls() failed'
    
    if passwd is None:
        passwd = getpass.getpass('password: ')
    rcode3, _ = smtp.login(sender, passwd)
    if rcode3 != 235:
        smtp.quit()
        return 'login failed'
    
    msg = MIMEText(message.encode('utf-8'), _subtype='plain', _charset='utf-8')
    msg['Subject'] = Header(subject.encode('utf-8'), 'utf-8')
    msg['From'] = myemail
    msg['To'] = ','.join(mail_to)

    smtp.sendmail(myemail, mail_to, msg.as_string( ))
    smtp.quit()

In [10]:
myemail = 'gslee0115@gmail.com'   # 내 주소
myemail = ('이강성', 'gslee0115@hanmail.net')   # 내 주소
receivers = [('이강성교수', 'gslee@mail.kw.ac.kr'), ('하늘길 대장', 'gslee111@naver.com')]  # 수신자의 메일 주소
subject = 'I love 파이썬'
message = '''
메일 시험 중..
파이썬으로 보내는 메일임..
'''

sendTextEmail(myemail, receivers, subject, message, )

password: ········


### 14.1.6 첨부 파일 메일 보내기

텍스트 파일 인코딩 자동 검출을 위하여 `chardet` 모듈을 설치한다. 텍스트 파일인 경우에는 인코딩을 정확하게 설정해야 제대로 된 문서를 발송할 수 있기 때문이다.

    pip install chardet

In [11]:
import smtplib
import mimetypes
import os
import re
import getpass
import chardet

import email
from email.header import Header
from email import encoders
from email.message import Message
from email.mime.base import MIMEBase
from email.mime.audio import MIMEAudio
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication


smtp_info = {      # SMTP 서버
    'gmail.com': ('smtp.gmail.com', 587),
    'naver.com': ('smtp.naver.com', 587),
    'hanmail.net': ('smtp.hanmail.net', 465),
    'nate.com': ('smtp.mail.nate.com', 465),
    'hotmail.com': ('smtp-mail.outlook.com', 587),
    'outlook.com': ('smtp-mail.outlook.com', 587),
    'yahoo.com': ('smtp.mail.yahoo.com', 587),
    }

email_pat = r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)"

def sendEmail(me, receivers, subject, message, attach_files=(), passwd=None):
    if type(me) is tuple:
        me = email.utils.formataddr(me)
    mail_to = []
    for receiver in receivers:
        if type(receiver) is tuple:
            receiver = email.utils.formataddr(receiver)
        mail_to.append(receiver)

    outer = MIMEBase('multipart', 'mixed')
    outer['Subject'] = Header(subject.encode('utf-8'), 'utf-8')
    outer['From'] = me
    outer['To'] = ', '.join(mail_to)   # 수신자 문자열 만들기
    outer.preamble = 'This is a multi-part message in MIME format.\n\n'
    outer.epilogue = ''	      # 이렇게 하면 멀티파트 경계 다음에 줄 바꿈 코드가 삽입 됨
    msg = MIMEText(message.encode('utf-8'), _charset='utf-8')
    outer.attach(msg)

    for fpath in attach_files:
        folder, fileName = os.path.split(fpath)
        ctype, encoding = mimetypes.guess_type(fileName)
        if ctype is None or encoding is not None:
            ctype = 'application/octet-stream'
        maintype, subtype = ctype.split('/', 1)
        if maintype == 'text':
            raw = open(fpath, 'rb').read()
            encoding = chardet.detect(raw)['encoding']
            msg = MIMEText(raw, _subtype=subtype, _charset=encoding)
        elif maintype == 'image':
            fd = open(fpath, 'rb')
            msg = MIMEImage(fd.read(), _subtype=subtype)
            fd.close()
        elif maintype == 'audio':
            fd = open(fpath, 'rb')
            msg = MIMEAudio(fd.read(), _subtype=subtype)
            fd.close()
        else:
            fd = open(fpath, 'rb')
            msg = MIMEApplication(fd.read(), _subtype=subtype)
            fd.close()
        msg.add_header('Content-Disposition', 'attachment', 
                       filename=(Header(fileName, 'utf-8').encode()))
        outer.attach(msg)

    # SMTP 서버를 통해서 메일 보내기
    sender = re.search(email_pat, me).groups()[0]
    _, host = sender.rsplit('@', 1)
    smtp_server, port = smtp_info[host]
    
    if port == 465:
        smtp = smtplib.SMTP_SSL(smtp_server, port)
        rcode1, _ = smtp.ehlo()
        rcode2 = 220
    elif port == 587:
        smtp = smtplib.SMTP(smtp_server, port)
        rcode1, _ = smtp.ehlo()
        rcode2, _ = smtp.starttls()
    else:
        return 'port {} not suported'.format(port)
        
    if rcode1 != 250:
        smtp.quit()
        return 'conection ehlo() failed'
    if rcode2 != 220:
        smtp.quit()
        return 'starttls() failed'

    if passwd is None:
        passwd = getpass.getpass('password: ')
    smtp.login(sender, passwd)
    smtp.sendmail(me, mail_to, outer.as_string())
    smtp.quit()

첨부파일 메일 보내기 테스트

In [12]:
me = 'gslee0115@gmail.com'         # 내 주소
me = ('이강성', 'gslee0115@hanmail.net')   # 내 주소
receivers = [('이강성교수', 'gslee@mail.kw.ac.kr'), ('하늘길 대장', 'gslee111@naver.com')]  # 수신자의 메일 주소

subject = '첨부 파일 메일 보내기 테스트'
message = '''
이 첨부파일은 한글 이름의 pdf 파일입니다.
잘 받았는지 확인 바랍니다.

감사합니다.
'''
attach_files = ['pdf/보고서_10월.pdf']
sendEmail(me, receivers, subject, message, attach_files, passwd='....')

## 14.2 이메일 받기

이메일을 가져오는 방법은 POP3를 이용하거나 IMAP4를 이용하는 방법이 있다.
먼저 만들어진 POP3는 서버에서 모든 메일을 읽어서 로컬 하드 디스크에 저장하고 서버에서 삭제한다. 
하지만 POP3는 메일 상자에 보관된 메일을 선택적으로 가져올 수 없다는 등의 문제점을 가지고 있다. 
이런 문제점을 해결하기 위해 IMAP4(Internet Message Access Protocol Version 4)가 등장한다. IMAP4의 기본 역할은 POP3와 동일하다. 그러나 IMAP4는 원격지 서버에서 메일을 관리하게하지만, 마치 자신의 컴퓨터에 있는 것처럼 서버의 메일 처럼 다룰 수 있게 해준다. 

### 14.2.1 준비

IMAP을 이용하려면 `imapclient`와 `pyzmail`이 필요하다. 다음과 같이 설치를 하자.

    pip install imapclient
    pip install pyzmail
    
다음 명령이 실행되면 설치가 잘 된 것이다.

In [13]:
import imapclient
import pyzmail

### 14.2.2 연결

각 업체별로 제공하는 IMAP 서버를 알아야 한다.

* 구글 : `imap.gmail.com`
* 네이버 : `imap.naver.com`
* 다음 : `imap.daum.net`

이 서버를 이용해 접속한다.

In [14]:
import imapclient
import pyzmail

imap = imapclient.IMAPClient('imap.gmail.com', ssl=True)

접속에 성공했으면 로그인을 한다.

In [15]:
imap.login('gslee0115@gmail.com', 'mypassword')

b'gslee0115@gmail.com authenticated (Success)'

### 14.2.3 폴더 선택

메일 폴더의 목록은 `list_folders()` 메서드로 확인한다.

In [16]:
imap.list_folders()

[((b'\HasNoChildren',), b'/', 'FairFaxChristianSchool'),
 ((b'\HasNoChildren',), b'/', 'INBOX'),
 ((b'\HasChildren', b'\Noselect'), b'/', '[Gmail]'),
 ((b'\Flagged', b'\HasNoChildren'), b'/', '[Gmail]/별표편지함'),
 ((b'\HasNoChildren', b'\Sent'), b'/', '[Gmail]/보낸편지함'),
 ((b'\HasNoChildren', b'\Junk'), b'/', '[Gmail]/스팸함'),
 ((b'\Drafts', b'\HasNoChildren'), b'/', '[Gmail]/임시보관함'),
 ((b'\All', b'\HasNoChildren'), b'/', '[Gmail]/전체보관함'),
 ((b'\HasNoChildren', b'\Important'), b'/', '[Gmail]/중요'),
 ((b'\HasNoChildren', b'\Trash'), b'/', '[Gmail]/휴지통')]


폴더를 하나 선택해야 한다.
`select_folder` 메서드의 `readonly` 인수가 `True`이면 메일을 읽어낼 수 있다.
`False`이면 읽기 및 삭제가 가능하다.
`readonly`가 `True`인 경우, 파이썬으로 메일을 읽어도 '읽은 메일' 표시되지 않는다.

In [17]:
imap.select_folder('INBOX', readonly=False)

{b'EXISTS': 3939,
 b'FLAGS': (b'\\Answered',
  b'\\Flagged',
  b'\\Draft',
  b'\\Deleted',
  b'\\Seen',
  b'$Forwarded',
  b'$NotPhishing',
  b'$Phishing'),
 b'HIGHESTMODSEQ': 1333177,
 b'PERMANENTFLAGS': (b'\\Answered',
  b'\\Flagged',
  b'\\Draft',
  b'\\Deleted',
  b'\\Seen',
  b'$Forwarded',
  b'$NotPhishing',
  b'$Phishing',
  b'\\*'),
 b'READ-WRITE': True,
 b'RECENT': 0,
 b'UIDNEXT': 9829,
 b'UIDVALIDITY': 596522249}

### 14.2.4 메일 검색

이제 메일 목록을 얻어보자.
`search()` 메서드를 이용해야 한다.

`imap.search()`

     'ALL' : 해당 폴더의 모든 메시지 목록
     'BEFORE 1-Feb-2011' : 지정된 날짜 이전의 메일들
     'ON 1-Feb-2011' : 지정된 날짜의 메일들
     'SINCE 1-Feb-2011' : 지정된 날짜부터의 메일들
     'SUBJECT 파이썬' : 제목으로 찾기
     'BODY 파이썬' : 본문으로 찾기
     'TEXT "파이썬 사무자동화"' : 제목 혹은 본문으로 찾기
     'FROM sender@some.com' : 발신자로 검색
     'TO receiver@some.com' : 수신자로 검색
     'CC receiver@some.com' : cc로 검색
     'BCC receiver@some.com' : bcc로 검색
     'SEEN' : 읽은 메일 목록
     'UNSEEN' : 읽지 않은 메일 목록
     'DELETED' : 삭제로 표시된 메일 목록
     'UNDELETED' : 삭제로 표시되지 않은 메일 목록
     'ANSWERED' : 응답한 메일 목록
     'UNANSWERED' : 응답하지 않은 메일 목록
     'DRAFT' : 임시 보관된 메일
     'UNDRAFT' : 
     'FLAGGED' : 플래그가 붙은 메일들
     'UNFLAGGED' : 플래그가 붙지 않은 메일들
     'LARGER 10000' : 전체 메시지 크기가 10000 보다 큰 메일들
     'SMALLER 10000' : 전체 메시지 크기가 10000 보다 작은 메일들
     'NOT search-key' : search-key와 일치하지 않는 메일들
     'OR search-key1 search-key2' : search-key1 혹은 search-key2와 일치하는 메일들


In [18]:
uids = imap.search(['SINCE 12-Nov-2015'])  # 

### 14.2.5 (Help)

만일 에러가 발생한다면 메모리 제한을 늘려야 한다.
기본 값은 10,000 바이트이다. 10,000,000 (10M) 바이트까지 늘릴 수 있다.

    import imaplib

    imaplib._MAXLINE   # 10000
    imaplib._MAXLINE = 10000000

만일 Gmail을 사용한다면 `gmail_search()` 메서드를 이용할 수도 있다.
Gmail에서 검색 하는 방식의 문자열 형식을 이용할 수 있다.

In [19]:
uids = imap.gmail_search('파이썬 사무자동화')

검색 결과는 메일의 아이디의 리스트이다.

In [20]:
uids

[9814, 9818, 9819]

### 14.2.6 메일 가져오기

이제 `fetch()` 메서드로 메일 내용을 가져와 보자.
`fetch()` 메서드는 여러 메일을 한 번에 읽을 수 있다. 
첫 인수는 아이디의 리스트이다.
두 번째 인수는 메일의 어떤 부분을 가져올지를 나타낸다.
`BODY[HEADER.FIELDS (SUBJECT FROM)]`는 메일에서 제목과 발신자의 내용을 가져온다.
`INTERNALDATE`는 메일이 도착한 시간을 가져온다.
`RFC822.SIZE`는 메일의 크기(바이트 수)를 가져온다.
`FLAGS`는 메일의 상태로 함께 가져오게 한다.
`fetch()`의 좀 더 자세한 내용은 https://tools.ietf.org/html/rfc1730 의 `fetch` 절을 참고한다.
다음 예는 `uids` 리스트의 전체 메일을 가져온다.

In [21]:
rmsgs = imap.fetch(uids, ['BODY[HEADER.FIELDS (SUBJECT FROM)]',
                          'INTERNALDATE',
                          'RFC822.SIZE', 
                          'FLAGS'])

반환 값을 확인해 보자.

In [22]:
for uid in uids:
    print(' ID:{}, {} bytes  FLAGS:{}'.format(uid, rmsgs[uid][b'RFC822.SIZE'], rmsgs[uid][b'FLAGS']))
    message = pyzmail.PyzMessage.factory(rmsgs[uid][b'BODY[HEADER.FIELDS (SUBJECT FROM)]'])
    print(message.get_addresses('from'), rmsgs[uid][b'INTERNALDATE'])
    print(message.get_subject())
    print()

 ID:9814, 274925 bytes  FLAGS:(b'\\Seen',)
[('K-ICT 빅데이터센터', 'bigdatamanager@nia.or.kr')] 2015-11-12 09:49:07
[BigData Monthly 제16호] 2015 Bigdata 수요기업 Trends 등

 ID:9818, 28943 bytes  FLAGS:(b'\\Seen',)
[('Google', 'no-reply@accounts.google.com')] 2015-11-12 16:55:16
Sign-in attempt prevented

 ID:9819, 27762 bytes  FLAGS:(b'\\Seen',)
[('Google', 'no-reply@accounts.google.com')] 2015-11-12 17:13:08
Access for less secure apps has been turned on



메일 내용은 `BODY[]`로 가져온다.

In [23]:
rmsgs = imap.fetch(uids, ['BODY[]'])

첫 번째 메일을 좀 더 탐구해 보자.
메일을 해석하는 방법은 몇 가지가 있지만 `pyzmail`을 이용하면 간편하다.

In [24]:
uid = uids[0]
message = pyzmail.PyzMessage.factory(rmsgs[uid][b'BODY[]'])

제목을 얻는다.

In [25]:
message.get_subject()

'[BigData Monthly 제16호] 2015 Bigdata 수요기업 Trends 등'

발신자를 얻는다.

In [26]:
message.get_addresses('from')

[('K-ICT 빅데이터센터', 'bigdatamanager@nia.or.kr')]

수신자를 얻는다.

In [27]:
message.get_addresses('to')

[('gslee0115@gmail.com', 'gslee0115@gmail.com')]

`CC, BCC`를 얻는다.

In [28]:
message.get_addresses('cc')

[]

In [29]:
message.get_addresses('bcc')

[]

메시지에 텍스트가 있는지 확인해본다.

In [30]:
message.text_part != None

False

만일 텍스트가 있을 경우는 다음 명령으로 해석한다.

    message.text_part.get_payload().decode(message.text_part.charset)
    
HTML 메일인지 확인한다.

In [31]:
message.html_part != None

True

HTML 문서를 가져온다.

In [32]:
html = message.html_part.get_payload().decode(message.html_part.charset)

In [33]:
html[:200]

'<title></title>\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\r\n\r\n\r\n<table border="0" cellpadding="0" cellspacing="0" width="700">\r\n<tbody><tr>\r\n\t<td height="15"></td>\r\n</tr>\r\n<t'

### 14.2.7 첨부 파일

다음 코드는 파일 이름이 있는 첨부 파일을 모두 다운 받는다.

In [34]:
for part in message.mailparts:
    if part.filename:
        print('saving..', part.filename, part.type)
        cont = part.get_payload()
        if part.type.startswith('text/'):
            open(part.filename, 'w').write(cont)
        else:
            open(part.filename, 'wb').write(cont)

saving.. Bigdata_Monthly_16.jpg image/jpeg


### 14.2.8 메일의 삭제
이메일의 삭제는 `delete_messages()`를 이용한다.
하지만 실제로 삭제되는 것은 아니고 삭제되었다는 표시만 해둔다.
실제의 삭제는 `expunge()` 메서드를 호출한다.
(Gmail인 경우는 `delete_messages()`로 실제로 삭제된다)

In [35]:
imap.delete_messages(uid)

{9812: (b'\\Seen', b'\\Deleted')}

`\Deleted`로 표시된 메일들을 영구히 삭제한다.

In [36]:
imap.expunge()

In [37]:
imap.logout()

b'LOGOUT Requested'

### 14.2.9 종합


In [38]:
import imapclient
import pyzmail
from bs4 import BeautifulSoup
import getpass

myemail = 'gslee0115@gmail.com'
passwd = getpass.getpass('password: ')

imap = imapclient.IMAPClient('imap.gmail.com', ssl=True)
imap.login(myemail, passwd)
imap.select_folder('INBOX', readonly=True)
uids = imap.search(['SINCE 12-Nov-2015'])   # 검색 조건 변경

for uid in uids:
    rmsgs = imap.fetch([uid], ['BODY[]', 'FLAGS'])
    message = pyzmail.PyzMessage.factory(rmsgs[uid][b'BODY[]'])
    subject = message.get_subject()
    senders = message.get_addresses('from')
    receivers = message.get_addresses('to')
    s_senders = ', '.join(['{}({})'.format(*sender) for sender in senders])
    s_receivers = ', '.join(['{}({})'.format(*receiver) for receiver in receivers])

    if message.text_part != None:
        text = message.text_part.get_payload().decode(message.text_part.charset)
    if message.html_part != None:
        html = message.html_part.get_payload().decode(message.html_part.charset)
        # html 문서에서 text 정보만 출력하는 부분
        soup = BeautifulSoup(html)
        for s in soup('script'):
            s.extract()    # script 태그를 분리한다.
        for s in soup('style'):
            s.extract()    # style 태그도 분리한다.
        text = '\n'.join(filter(bool, soup.text.strip().splitlines()))

    email_msg = '''
    ------------------------------------------------------------------------
    [{}] {}
    ------------------------------------------------------------------------
    {} ==> {}

    {}
    '''.format(uid, subject, s_senders, s_receivers, text)
    print(email_msg)

    # 첨부 파일 저장
    for part in message.mailparts:
        if part.filename:
            print('saving..', part.filename, part.type)
            cont = part.get_payload()
            if part.type.startswith('text/'):
                open(part.filename, 'w').write(cont)
            else:
                open(part.filename, 'wb').write(cont)
            print ('Attached file', part.filename, 'saved..')    
            
imap.logout();

password: ········
    ------------------------------------------------------------------------
    [9814] [BigData Monthly 제16호] 2015 Bigdata 수요기업 Trends 등
    ------------------------------------------------------------------------
    K-ICT 빅데이터센터(bigdatamanager@nia.or.kr) ==> gslee0115@gmail.com(gslee0115@gmail.com)

    K-ICT 빅데이터센터
URL. http://www.kbig.kr
ADDR. 대구광역시 동구 첨단로 53 한국정보화진흥원
Copyright  2014  http://www.kbig.kr   All Rights Reserved
    
Attached file [9814]Bigdata_Monthly_16.jpg saved..
